
**NOTE THAT THIS IS NOT A TRAINED MODEL DUE TO HARDWARE REQUIREMENTS AND A VERY INEFFICIENT APPROACH**

This code was not used in the actual model in production. It was only a demonstration of another way to process data withous using data generator of keras. 


## Setup and imports 


In [1]:
# Import the required libraries and the modules.

import cv2
import os
import tqdm
import numpy as np
from tqdm import tqdm
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from google.colab import drive          
drive.mount('/content/drive')

Mounted at /content/drive


## Data Loading and pre-processing

In [4]:
# Function to load images from a folder

def load_images_from_folder(folder):
    images = []
    for filename in tqdm(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder,filename))
        imgResized = cv2.resize(img, (128,128))
        imgResized = imgResized.astype('int')
        imgResized = imgResized.tolist()
        if img is not None:
            images.append(imgResized)
    return images


# We have used a dummy folder containing only 4 images.
# This is because it is not possible to take all the images into a list due to RAM requirements
folder_dummy = "/content/drive/MyDrive/Datasets/dummy_mech_parts"

# This is the actual folders containing the images
folder1="/content/drive/MyDrive/Datasets/mechanical_parts_dataset/training/bolt"
folder2="/content/drive/MyDrive/Datasets/mechanical_parts_dataset/training/locatingpin"
folder3="/content/drive/MyDrive/Datasets/mechanical_parts_dataset/training/nut"
folder4="/content/drive/MyDrive/Datasets/mechanical_parts_dataset/training/washer"

In [6]:
xtrain_dummy = load_images_from_folder(folder_dummy)

len(xtrain_dummy)

100%|██████████| 4/4 [00:00<00:00, 34.35it/s]


4

In [7]:
xtrain_dummy[:2]

[[[[255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],
   [255, 255, 255],


In [13]:
# Convert the list to array as the model accpts only arrays as swift calculations can be performed. 
xtrain_dummy = np.array(xtrain_dummy)

In [32]:
# Understanding the shape of our input array i.e. array of images
xtrain_dummy.shape

(4, 128, 128, 3)

In [ ]:
# To be run for actual training

xtrain1 = load_images_from_folder(folder1)
xtrain2 = load_images_from_folder(folder2)
xtrain3 = load_images_from_folder(folder3)
xtrain4 = load_images_from_folder(folder4)

 36%|███▌      | 1900/5282 [00:59<49:58,  1.13it/s] 

In [ ]:
# To be run for actual training

x_train = []
x_train.append(xtrain1)
x_train.append(xtrain2)
x_train.append(xtrain3)
x_train.append(xtrain4)
x_train=np.array(x_train)

In [ ]:
# path of folders for testing data

folder5="/content/drive/MyDrive/Datasets/dataset/testing/bolt"
folder6="/content/drive/MyDrive/Datasets/dataset/testing/locatingpin"
folder7="/content/drive/MyDrive/Datasets/dataset/testing/nut"
folder8="/content/drive/MyDrive/Datasets/dataset/testing/washer"

In [ ]:
# Loading testing data

xtest1 = load_images_from_folder(folder5)
xtest2 = load_images_from_folder(folder6)
xtest3 = load_images_from_folder(folder7)
xtest4 = load_images_from_folder(folder8)

In [ ]:
# Taking testing data into a single list and then converting it to array

x_test=[]
x_test.append(xtest1)
x_test.append(xtest2)
x_test.append(xtest3)
x_test.append(xtest4)
x_test = np.array(x_test)

In [ ]:
# Only to be run while actual training
# Preparing one hot encoding of the training data

y_train = np.zeros((x_train.shape[0]*x_train.shape[1],4))
for i in range (x_train.shape[0]*x_train.shape[1]):
  j = i//x_train.shape[1]
  y_train[i][j] = 1

In [ ]:
# Preparing one hot encoding of the testing data

y_test = np.zeros((x_test.shape[0]*x_test.shape[1],4))
for i in range (x_test.shape[0]*x_test.shape[1]):
  j = i//x_test.shape[1]
  y_test[i][j] = 1

In [15]:
# Preparing dummy y_train for demonstration

ytrain_dummy = np.zeros((4,4))
for i in range (4):
  ytrain_dummy[i][0] = 1

In [19]:
print(xtrain_dummy.shape)
print(ytrain_dummy.shape)

(4, 128, 128, 3)
(4, 4)


## Model and Training

In [16]:
# This is our CNN-based model

def MechanicalModel():

  X_input = Input((128,128,3))
  X = ZeroPadding2D((3,3))(X_input)
  X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
  X = BatchNormalization(axis = 3, name = 'bn0')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool0')(X)
  
  X = ZeroPadding2D((3,3))(X)
  X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv1')(X)
  X = BatchNormalization(axis = 3, name = 'bn1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool1')(X)
  
  X = ZeroPadding2D((3,3))(X)
  X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv2')(X)
  X = BatchNormalization(axis = 3, name = 'bn2')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool2')(X)

  X = ZeroPadding2D((3,3))(X)
  X = Conv2D(128, (7, 7), strides = (1, 1), name = 'conv3')(X)
  X = BatchNormalization(axis = 3, name = 'bn3')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool3')(X)
  
  X = Flatten()(X)
  X = Dense(4, activation = 'softmax', name = 'fc')(X)
  print(X)
  model = tf.keras.Model(inputs = X_input, outputs = X, name = 'MechanicalModel')

  return model

In [20]:
# We compile it and apply the loss function and the optimizer
# Note that categorical_crossentropy is used since the y values are in one-hot encoded form. 

mechanicalmodel = MechanicalModel()

mechanicalmodel.compile(optimizer = 'adam', loss = tf.keras.losses.categorical_crossentropy, metrics = ["accuracy"])

KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='fc/Softmax:0', description="created by layer 'fc'")


In [21]:
# We train the model on only 4 images as this approach is memory inefficient 

model = mechanicalmodel.fit(xtrain_dummy, ytrain_dummy, epochs=50, batch_size=4)

Epoch 1/50
1/1 [==============================] - 7s 7s/step - loss: 1.7688 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy

In [ ]:
# The predictions 

preds = mechanicalmodel.evaluate(x = x_test, y = y_test)
print ("Loss = " + str(preds[0]))
print ("Validation Accuracy = " + str(preds[1]))